# Flexibility Analysis

## Algorithm to Automate Flexibility Scoring

### Import Packages

In [1]:
import nltk
import pandas as pd
import numpy as np

import os
from os.path import expanduser
import glob

from scipy import stats
from scipy.stats import pearsonr

import shared_functions as sf
from shared_functions import *

import originality_methods as om
from originality_methods import  *

import flexibility_methods as fm
from flexibility_methods import *

## Read in Data to Analyze

In [2]:
cwd = os.getcwd()

In [28]:
# read in the official novelty results
novelty_dict = {}
for filename in glob.glob(cwd + '/..//results/results_methods/novelty_results/novelty_071921/*.csv'):
    novelty_dict[filename[122:-4]] = pd.read_csv(filename)

In [29]:
# read in the official flexibility results
flexibility_dict = {}
for filename in glob.glob(cwd + '/..//results/results_methods/flexibility_results/flexibility_071921/*.csv'):
    flexibility_dict[filename[157:-4]] = pd.read_csv(filename)

In [30]:
# read in the official originality results
originality_dict = {}
for filename in glob.glob(cwd + '/..//results/results_methods/originality_results/originality_071921/freqs/*.csv'):
    originality_dict[filename[169:-4]] = pd.read_csv(filename)

In [31]:
# read in the official fluency results
fluency_dict = {}
for filename in glob.glob(cwd + '/..//results/results_methods/fluency_results/fluency_071521/*.csv'):
    fluency_dict[filename[145:-4]] = pd.read_csv(filename)

## Comparing Flexibility Results with Human Ratings

In [7]:
# create pvalues matrix
def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

### Correlations for Each Prompt
#### Between Human Rater and Method

In [32]:
# calculate corrrelation once
def flexibility_corr(flexibility_dict):
    # list to store list of tuples, later turn into df
    corrs_tuple_list = []
    # get list of keys to iterate through
    flexibility_keys = list(flexibility_dict.keys())
    # calculate the corr coefficient and pval
    for data in flexibility_keys:
        corrs_tuple_list.append(stats.pearsonr(flexibility_dict[data]['flex_m'], flexibility_dict[data]['flex_method']))
        
    # turn list of tuples into df
    result_df = pd.DataFrame(corrs_tuple_list, columns=['corrs', 'pval'])
    
    # rename index with corresponding prompt
    result_df.index = flexibility_keys
    
    # return df
    return result_df

In [33]:
flexibility_corr(flexibility_dict)

,corrs,pval
box,0.397504,0.000115
brick,0.277716,0.008414
chair,0.222167,0.035328
cup,0.168713,0.113996
key,0.421136,0.000036
pencil,0.281232,0.007250
rope,0.336711,0.001175
shoe,0.308043,0.003140


In [11]:
def get_novelty_avg_semdis(collapse_df):
    # get id list
    id_list = sf.get_id_list(collapse_df)
    participant_values = []
    
    for participant in id_list:
        values_list = []
        values_list.append(participant)
        temp_df = collapse_df.loc[collapse_df['id'] == participant]
        values_list.append(temp_df["ewm_vector_cosine_dis"].mean())
        values_list.append(temp_df["ewm_vector_cosine_dis_clus_avg"].mean())
        values_list.append(temp_df["minima_vector_cosine_dis"].mean())
        values_list.append(temp_df["minima_vector_cosine_dis_clus_avg"].mean())
        values_list.append(temp_df["minima_vector_cosine_dis_clus_min"].mean())
        values_list.append(temp_df["SemDis_cbowukwacsubtitle_nf_m"].mean())
        values_list.append(temp_df["SemDis_cbowsubtitle_nf_m"].mean())
        values_list.append(temp_df["SemDis_cbowBNCwikiukwac_nf_m"].mean())
        values_list.append(temp_df["SemDis_TASA_nf_m"].mean())
        values_list.append(temp_df["SemDis_glove_nf_m"].mean())
        values_list.append(temp_df["SemDis_MEAN"].mean())
        values_list.append(temp_df["novelty_1"].mean())
        values_list.append(temp_df["novelty_2"].mean())
        values_list.append(temp_df["novelty_m"].mean())

        values_tuple = tuple(values_list)
        participant_values.append(values_tuple)
        
    participant_avg_novelty_df = pd.DataFrame(participant_values, columns=['id', 'ewm', 'ewm_clust', 'minvec', 'minvec_clust', "minvec_min",
                                                                          'cbowukwacsubtitle', 'cbowsubtitle', 'cbowBNCwikiukwac', 'TASA', 'glove'
                                                                           , "SemDis_MEAN", 'novelty_1', 'novelty_2', 'novelty_m'])
    return participant_avg_novelty_df

In [51]:
def get_novelty_avg(collapse_df):
    # get id list
    id_list = sf.get_id_list(collapse_df)
    participant_values = []
    
    for participant in id_list:
        values_list = []
        values_list.append(participant)
        temp_df = collapse_df.loc[collapse_df['id'] == participant]
        values_list.append(temp_df["ewm_vector_cosine_dis"].mean())
        values_list.append(temp_df["ewm_vector_cosine_dis_clus_avg"].mean())
        values_list.append(temp_df["minima_vector_cosine_dis"].mean())
        values_list.append(temp_df["minima_vector_cosine_dis_clus_avg"].mean())
        values_list.append(temp_df["minima_vector_cosine_dis_clus_min"].mean())
        values_list.append(temp_df["novelty_1"].mean())
        values_list.append(temp_df["novelty_2"].mean())
        values_list.append(temp_df["novelty_m"].mean())
        values_tuple = tuple(values_list)
        participant_values.append(values_tuple)
        
    participant_avg_novelty_df = pd.DataFrame(participant_values, columns=['id', 'ewm', 'ewm_clust', 'minvec'
                                                                           , 'minvec_clust', "minvec_min",
                                                                           "novelty_1", 'novelty_2', 'novelty_m'])
    
    return participant_avg_novelty_df

In [52]:
def get_flexibility_avg(collapse_df):
    # get id list
    id_list = sf.get_id_list(collapse_df)
    participant_values = []
    
    for participant in id_list:
        values_list = []
        values_list.append(participant)
        temp_df = collapse_df.loc[collapse_df['id'] == participant]
        values_list.append(temp_df["flex_1"].mean())
        values_list.append(temp_df["flex_2"].mean())
        values_list.append(temp_df["flex_m"].mean())
        values_list.append(temp_df["flex_method"].mean())
        values_tuple = tuple(values_list)
        participant_values.append(values_tuple)
        
    participant_avg_flexibility_df = pd.DataFrame(participant_values, columns=['id', 'flex_1', 'flex_2', "flex_m", 'flex_method'])
    
    return participant_avg_flexibility_df

In [53]:
def get_originality_avg(collapse_df):
    # get id list
    id_list = sf.get_id_list(collapse_df)
    participant_values = []
    
    for participant in id_list:
        values_list = []
        values_list.append(participant)
        temp_df = collapse_df.loc[collapse_df['id'] == participant]
        values_list.append(temp_df["t_freq"].mean())
        values_tuple = tuple(values_list)
        participant_values.append(values_tuple)
        
    participant_avg_originality_df = pd.DataFrame(participant_values, columns=['id', 'originality'])
    
    return participant_avg_originality_df

In [54]:
def get_fluency_avg(collapse_df):
    # get id list
    id_list = sf.get_id_list(collapse_df)
    participant_values = []
    
    for participant in id_list:
        values_list = []
        values_list.append(participant)
        temp_df = collapse_df.loc[collapse_df['id'] == participant]
        values_list.append(temp_df["fluency"].mean())
        values_tuple = tuple(values_list)
        participant_values.append(values_tuple)
        
    participant_avg_fluency_df = pd.DataFrame(participant_values, columns=['id', 'fluency'])
    
    return participant_avg_fluency_df

## Big Correlation Matrix

In [40]:
def calculate_mega_matrix(novelty_dict_values, flexibility_dict_values, originality_dict_value, fluency_dict_value):
    collapse_novelty_df = pd.concat(novelty_dict_values)
    collapse_flexibility_df = pd.concat(flexibility_dict_values)
    collapse_originality_df = pd.concat(originality_dict_value)
    collapse_fluency_df = pd.concat(fluency_dict_value)
    participant_avg_novelty_df = get_novelty_avg_semdis(collapse_novelty_df)
    participant_avg_flexibility_df = get_flexibility_avg(collapse_flexibility_df)
    participant_avg_originality_df = get_originality_avg(collapse_originality_df)
    participant_avg_fluency_df = get_fluency_avg(collapse_fluency_df)
    combined_df = pd.merge(participant_avg_novelty_df, participant_avg_flexibility_df, how='inner', on = 'id')
    combined_df = pd.merge(combined_df, participant_avg_originality_df, how='inner', on = 'id')
    combined_df = pd.merge(combined_df, participant_avg_fluency_df, how='inner', on = 'id')
    combined_df = combined_df.drop('id', axis=1)
    pval_df = calculate_pvalues(combined_df)
    corrs_df = combined_df.corr()
    return (corrs_df, pval_df)

In [45]:
mega_matrix = calculate_mega_matrix(novelty_dict.values(), flexibility_dict.values(), originality_dict.values(), fluency_dict.values())

In [46]:
# mega_matrix[0].to_csv("mega_matrix_corrs_071921.csv")

In [47]:
# mega_matrix[1].to_csv("mega_matrix_pvals_071921.csv")

## Combine Semantic Spaces
## Per Prompt per Individual

In [64]:
novel_dict_box = ['box_banori_results', 'box_cbow_6_ukwac_subtitle_results', 'box_cbow_subtitle_results', 'box_glove_6B_results', 'box_TASA_results']
novel_dict_brick = ['brick_banori_results', 'brick_cbow_6_ukwac_subtitle_results', 'brick_cbow_subtitle_results', 'brick_glove_6B_results', 'brick_TASA_results']
novel_dict_chair = ['chair_banori_results', 'chair_cbow_6_ukwac_subtitle_results', 'chair_cbow_subtitle_results', 'chair_glove_6B_results', 'chair_TASA_results']
novel_dict_cup = ['cup_banori_results', 'cup_cbow_6_ukwac_subtitle_results', 'cup_cbow_subtitle_results', 'cup_glove_6B_results', 'cup_TASA_results']
novel_dict_key = ['key_banori_results', 'key_cbow_6_ukwac_subtitle_results', 'key_cbow_subtitle_results', 'key_glove_6B_results', 'key_TASA_results']
novel_dict_pencil = ['pencil_banori_results', 'pencil_cbow_6_ukwac_subtitle_results', 'pencil_cbow_subtitle_results', 'pencil_glove_6B_results', 'pencil_TASA_results']
novel_dict_rope = ['rope_banori_results', 'rope_cbow_6_ukwac_subtitle_results', 'rope_cbow_subtitle_results', 'rope_glove_6B_results', 'rope_TASA_results']
novel_dict_shoe = ['shoe_banori_results', 'shoe_cbow_6_ukwac_subtitle_results', 'shoe_cbow_subtitle_results', 'shoe_glove_6B_results', 'shoe_TASA_results']

In [70]:
novel_dict_list = [novel_dict_box, novel_dict_brick, novel_dict_chair, novel_dict_cup, novel_dict_key
                  , novel_dict_pencil, novel_dict_rope, novel_dict_shoe]

In [65]:
def collapse_prompt_novelty(prompt_list):
    df_list = []
    for key in prompt_list:
        df_list.append(nobbvelty_dict[key])
        
    return pd.concat(df_list)

In [80]:
def print_per_prompt_results(metric_dict, function):
    key_list = list(metric_dict.keys())
    print(function)
    for i in range(len(key_list)):
        print(key_list[i])
        function(novel_dict_list[i], key_list[i])

## Comparing Flexibility Results with Novelty Results

In [103]:
def calculate_corrs_novelty_flexibility(prompt_list, prompt):
    collapse_novelty_df = collapse_prompt_novelty(prompt_list)
    collapse_flexibility_df = flexibility_dict[prompt]
    participant_avg_novelty_df = get_novelty_avg(collapse_novelty_df)
    participant_avg_flexibility_df = get_flexibility_avg(collapse_flexibility_df)
    combined_df = pd.merge(participant_avg_novelty_df, participant_avg_flexibility_df, how='inner', on = 'id')
    combined_df = combined_df.drop('id', axis=1)
    pval_df = calculate_pvalues(combined_df)
    corrs_df = combined_df.corr()
    corrs_df = pd.DataFrame(corrs_df.iloc[:8, 8:])
    pval_df = pd.DataFrame(pval_df.iloc[:8, 8:])
    pval_df.columns = ['flex_1_pval', 'flex_2_pval', 'flex_m_pval', 'flex_method_pval']
    corr_pval_df = pd.concat([corrs_df, pval_df], axis=1)
    corr_pval_df = corr_pval_df[['flex_1', 'flex_1_pval','flex_2', 'flex_2_pval', 'flex_m', 'flex_m_pval', 'flex_method', 'flex_method_pval']]
    corr_pval_df.index.rename('metrics', inplace=True)
    display(corr_pval_df)

In [104]:
print_per_prompt_results(flexibility_dict, calculate_corrs_novelty_flexibility)

<function calculate_corrs_novelty_flexibility at 0x00000173B589D0D0>
box


,flex_1,flex_1_pval,flex_2,flex_2_pval,flex_m,flex_m_pval,flex_method,flex_method_pval
metrics,,,,,,,,
ewm,0.196274,0.0653,0.196274,0.0653,0.196274,0.0653,0.089172,0.406
ewm_clust,0.097353,0.3641,0.097353,0.3641,0.097353,0.3641,-0.035171,0.7435
minvec,0.065280,0.5433,0.065280,0.5433,0.065280,0.5433,0.200279,0.0599
minvec_clust,0.139266,0.193,0.139266,0.193,0.139266,0.193,0.229530,0.0305
minvec_min,-0.084527,0.4309,-0.084527,0.4309,-0.084527,0.4309,0.059106,0.5822
novelty_1,0.399489,0.0001,0.399489,0.0001,0.399489,0.0001,0.305716,0.0036
novelty_2,0.428859,0.0,0.428859,0.0,0.428859,0.0,0.171730,0.1076
novelty_m,0.455421,0.0,0.455421,0.0,0.455421,0.0,0.259389,0.0141


brick


,flex_1,flex_1_pval,flex_2,flex_2_pval,flex_m,flex_m_pval,flex_method,flex_method_pval
metrics,,,,,,,,
ewm,0.054062,0.6148,0.054062,0.6148,0.054062,0.6148,0.355772,0.0006
ewm_clust,-0.046324,0.6664,-0.046324,0.6664,-0.046324,0.6664,0.258929,0.0143
minvec,0.182371,0.0872,0.182371,0.0872,0.182371,0.0872,0.331286,0.0015
minvec_clust,0.062480,0.5608,0.062480,0.5608,0.062480,0.5608,0.268980,0.0108
minvec_min,-0.380801,0.0002,-0.380801,0.0002,-0.380801,0.0002,0.150023,0.1605
novelty_1,0.354233,0.0007,0.354233,0.0007,0.354233,0.0007,0.142514,0.1828
novelty_2,0.267516,0.0113,0.267516,0.0113,0.267516,0.0113,-0.023866,0.8243
novelty_m,0.344863,0.0009,0.344863,0.0009,0.344863,0.0009,0.064001,0.5513


chair


...

,flex_1,flex_1_pval,flex_2,flex_2_pval,flex_m,flex_m_pval,flex_method,flex_method_pval
metrics,,,,,,,,
ewm,0.373547,0.0003,0.373547,0.0003,0.373547,0.0003,0.128012,0.2292
ewm_clust,0.370203,0.0003,0.370203,0.0003,0.370203,0.0003,0.293901,0.0049
minvec,0.404287,0.0001,0.404287,0.0001,0.404287,0.0001,0.091054,0.3934
minvec_clust,0.413553,0.0001,0.413553,0.0001,0.413553,0.0001,0.372617,0.0003
minvec_min,-0.067481,0.5274,-0.067481,0.5274,-0.067481,0.5274,0.141753,0.1826
novelty_1,0.419946,0.0,0.419946,0.0,0.419946,0.0,0.113111,0.2885
novelty_2,0.431016,0.0,0.431016,0.0,0.431016,0.0,0.194563,0.0661
novelty_m,0.444530,0.0,0.444530,0.0,0.444530,0.0,0.159341,0.1336


rope


,flex_1,flex_1_pval,flex_2,flex_2_pval,flex_m,flex_m_pval,flex_method,flex_method_pval
metrics,,,,,,,,
ewm,-0.109186,0.3056,-0.109186,0.3056,-0.109186,0.3056,-0.068825,0.5192
ewm_clust,0.031529,0.768,0.031529,0.768,0.031529,0.768,0.109411,0.3046
minvec,0.133046,0.2113,0.133046,0.2113,0.133046,0.2113,0.034482,0.747
minvec_clust,0.159281,0.1337,0.159281,0.1337,0.159281,0.1337,0.197240,0.0624
minvec_min,-0.185170,0.0806,-0.185170,0.0806,-0.185170,0.0806,0.219454,0.0377
novelty_1,0.242057,0.0215,0.242057,0.0215,0.242057,0.0215,-0.007393,0.9449
novelty_2,0.367630,0.0004,0.367630,0.0004,0.367630,0.0004,-0.104984,0.3247
novelty_m,0.329277,0.0015,0.329277,0.0015,0.329277,0.0015,-0.058887,0.5814


shoe


,flex_1,flex_1_pval,flex_2,flex_2_pval,flex_m,flex_m_pval,flex_method,flex_method_pval
metrics,,,,,,,,
ewm,-0.007673,0.9428,-0.007673,0.9428,-0.007673,0.9428,0.018983,0.859
ewm_clust,-0.172019,0.105,-0.172019,0.105,-0.172019,0.105,0.099337,0.3516
minvec,0.015195,0.887,0.015195,0.887,0.015195,0.887,0.064706,0.5446
minvec_clust,-0.115987,0.2763,-0.115987,0.2763,-0.115987,0.2763,0.069095,0.5176
minvec_min,-0.067786,0.5255,-0.067786,0.5255,-0.067786,0.5255,0.483804,0.0
novelty_1,0.084244,0.4299,0.084244,0.4299,0.084244,0.4299,-0.019454,0.8556
novelty_2,0.137147,0.1974,0.137147,0.1974,0.137147,0.1974,0.039699,0.7103
novelty_m,0.116263,0.2752,0.116263,0.2752,0.116263,0.2752,0.010683,0.9204


ewm:
- 2/8 sig flex_1, flex_2, flex_m, flex_method

ewm_clust:
- 3/8 sig flex_1, flex_2, flex_m
- 4/8 sig flex_method

minvec:
- 3/8 sig flex_1, flex_2, flex_m

minvec_clust:
- 3/8 sig flex_1, flex_m, flex_m
- 4/8 sig flex_method

minvec_min:
- 2/8 sig flex_1, flex_2, flex_m
- 4/8 sig flex_method

novelty_1:
- 7/8 sig flex_1, flex_2, flex_m
- 1/8 sig flex_method

novelty_2:
- 7/8 sig flex_1, flex_2, flex_m
- 3/8 sig flex_method

novelty_3:
- 7/8 sig flex_1, flex_2, flex_m
- 2/8 sig flex_method

## Comparing Originality with Novelty Results

In [105]:
def calculate_corrs_novelty_originality(prompt_list, prompt):
    collapse_novelty_df = collapse_prompt_novelty(prompt_list)
    collapse_originality_df = originality_dict[prompt]
    participant_avg_novelty_df = get_novelty_avg(collapse_novelty_df)
    participant_avg_originality_df = get_originality_avg(collapse_originality_df)
    combined_df = pd.merge(participant_avg_novelty_df, participant_avg_originality_df, how='inner', on = 'id')
    combined_df = combined_df.drop('id', axis=1)
    pval_df = calculate_pvalues(combined_df)
    corrs_df = combined_df.corr()
    corrs_df = pd.DataFrame(corrs_df.iloc[:8, 8:])
    pval_df = pd.DataFrame(pval_df.iloc[:8, 8:])
    pval_df.columns = ['originality_pval']
    corr_pval_df = pd.concat([corrs_df, pval_df], axis=1)
    corr_pval_df = corr_pval_df[['originality', 'originality_pval']]
    corr_pval_df.index.rename('metrics', inplace=True)
    display(corr_pval_df)

In [106]:
print_per_prompt_results(originality_dict, calculate_corrs_novelty_originality)

<function calculate_corrs_novelty_originality at 0x00000173B589D670>
box


,originality,originality_pval
metrics,,
ewm,0.189612,0.0751
ewm_clust,-0.000058,0.9996
minvec,-0.000268,0.998
minvec_clust,-0.147569,0.1676
minvec_min,0.530717,0.0
novelty_1,-0.265684,0.0119
novelty_2,-0.322260,0.0021
novelty_m,-0.323983,0.002


brick


,originality,originality_pval
metrics,,
ewm,0.062293,0.562
ewm_clust,0.048276,0.6532
minvec,-0.018245,0.8652
minvec_clust,0.024448,0.8201
minvec_min,0.194064,0.0684
novelty_1,-0.192922,0.0701
novelty_2,-0.018384,0.8642
novelty_m,-0.115453,0.2813


chair


...

,originality,originality_pval
metrics,,
ewm,0.371915,0.0003
ewm_clust,0.532983,0.0
minvec,0.119971,0.26
minvec_clust,0.415412,0.0
minvec_min,0.150080,0.158
novelty_1,0.145888,0.1701
novelty_2,0.166577,0.1166
novelty_m,0.162930,0.1249


rope


,originality,originality_pval
metrics,,
ewm,0.366624,0.0004
ewm_clust,0.316544,0.0024
minvec,0.232547,0.0274
minvec_clust,0.151000,0.1554
minvec_min,0.298730,0.0042
novelty_1,-0.123427,0.2465
novelty_2,0.020861,0.8453
novelty_m,-0.059404,0.5781


shoe


,originality,originality_pval
metrics,,
ewm,0.151010,0.1554
ewm_clust,0.104894,0.3252
minvec,0.141800,0.1825
minvec_clust,0.170178,0.1088
minvec_min,0.172933,0.1031
novelty_1,0.037155,0.7281
novelty_2,-0.042322,0.6921
novelty_m,-0.002787,0.9792


novelty_m and novelt_2:
- 2/8 sig

novelty_1:
- 1/8 sig

ewm:
- 3/8 sig

ewm_clust:
- 5/8 sig

minvec:
- 2/8 sig

minvec_clust:
- 3/8 sig

minvec_min:
- 4/8 sig


## Comparing Originality Results with Flexibility

In [118]:
def calculate_corrs_originality_flexibility(prompt_list, prompt):
    collapse_flexibility_df = flexibility_dict[prompt]
    collapse_originality_df = originality_dict[prompt]
    participant_avg_flexibility_df = get_flexibility_avg(collapse_flexibility_df)
    participant_avg_originality_df = get_originality_avg(collapse_originality_df)
    combined_df = pd.merge(participant_avg_flexibility_df, participant_avg_originality_df, how='inner', on = 'id')
    combined_df = combined_df.drop('id', axis=1)
    pval_df = calculate_pvalues(combined_df)
    corrs_df = combined_df.corr()
    corrs_df = pd.DataFrame(corrs_df.iloc[:4, 4:])
    pval_df = pd.DataFrame(pval_df.iloc[:4, 4:])
    pval_df.columns = ['originality_pval']
    corr_pval_df = pd.concat([corrs_df, pval_df], axis=1)
    corr_pval_df = corr_pval_df[['originality', 'originality_pval']]
    corr_pval_df.index.rename('metrics', inplace=True)
    display(corr_pval_df)

In [119]:
print_per_prompt_results(originality_dict, calculate_corrs_originality_flexibility)

<function calculate_corrs_originality_flexibility at 0x00000173B5B57C10>
box


,originality,originality_pval
metrics,,
flex_1,-0.127229,0.2348
flex_2,-0.127229,0.2348
flex_m,-0.127229,0.2348
flex_method,-0.142775,0.182


brick


,originality,originality_pval
metrics,,
flex_1,-0.085636,0.4249
flex_2,-0.085636,0.4249
flex_m,-0.085636,0.4249
flex_method,0.064088,0.5507


chair


...

,originality,originality_pval
metrics,,
flex_1,0.143285,0.1779
flex_2,0.143285,0.1779
flex_m,0.143285,0.1779
flex_method,0.276725,0.0083


rope


,originality,originality_pval
metrics,,
flex_1,-0.155248,0.144
flex_2,-0.155248,0.144
flex_m,-0.155248,0.144
flex_method,0.052038,0.6262


shoe


,originality,originality_pval
metrics,,
flex_1,-0.008074,0.9398
flex_2,-0.008074,0.9398
flex_m,-0.008074,0.9398
flex_method,0.220326,0.0369


human ratings:
 - never sig
 
flex_method:
 - 4/8 sig

## Comparing Fluency with Novelty Results

In [107]:
def calculate_corrs_novelty_fluency(prompt_list, prompt):
    collapse_novelty_df = collapse_prompt_novelty(prompt_list)
    collapse_fluency_df = fluency_dict[prompt]
    participant_avg_novelty_df = get_novelty_avg(collapse_novelty_df)
    participant_avg_fluency_df = get_fluency_avg(collapse_fluency_df)
    combined_df = pd.merge(participant_avg_novelty_df, participant_avg_fluency_df, how='inner', on = 'id')
    combined_df = combined_df.drop('id', axis=1)
    pval_df = calculate_pvalues(combined_df)
    corrs_df = combined_df.corr()
    corrs_df = pd.DataFrame(corrs_df.iloc[:8, 8:])
    pval_df = pd.DataFrame(pval_df.iloc[:8, 8:])
    pval_df.columns = ['fluency_pval']
    corr_pval_df = pd.concat([corrs_df, pval_df], axis=1)
    corr_pval_df = corr_pval_df[['fluency', 'fluency_pval']]
    corr_pval_df.index.rename('metrics', inplace=True)
    display(corr_pval_df)

In [108]:
print_per_prompt_results(fluency_dict, calculate_corrs_novelty_fluency)

<function calculate_corrs_novelty_fluency at 0x00000173B589D3A0>
box


,fluency,fluency_pval
metrics,,
ewm,0.034228,0.7502
ewm_clust,0.080534,0.4531
minvec,0.110455,0.3028
minvec_clust,0.121902,0.2551
minvec_min,-0.236841,0.0254
novelty_1,0.222566,0.036
novelty_2,0.355715,0.0006
novelty_m,0.320255,0.0022


brick


,fluency,fluency_pval
metrics,,
ewm,0.145437,0.1739
ewm_clust,0.101424,0.3443
minvec,0.263180,0.0127
minvec_clust,0.340491,0.0011
minvec_min,-0.181089,0.0894
novelty_1,0.456836,0.0
novelty_2,0.329723,0.0016
novelty_m,0.436067,0.0


chair


...

,fluency,fluency_pval
metrics,,
ewm,0.242698,0.0212
ewm_clust,0.311347,0.0028
minvec,0.290443,0.0055
minvec_clust,0.368023,0.0004
minvec_min,-0.216918,0.04
novelty_1,0.310298,0.0029
novelty_2,0.338503,0.0011
novelty_m,0.338570,0.0011


rope


,fluency,fluency_pval
metrics,,
ewm,-0.002943,0.978
ewm_clust,0.217052,0.0399
minvec,0.173132,0.1027
minvec_clust,0.321822,0.002
minvec_min,-0.201661,0.0566
novelty_1,0.254074,0.0157
novelty_2,0.175493,0.098
novelty_m,0.236050,0.0251


shoe


,fluency,fluency_pval
metrics,,
ewm,-0.037727,0.7241
ewm_clust,-0.074997,0.4823
minvec,0.072376,0.4978
minvec_clust,0.100284,0.347
minvec_min,-0.130148,0.2215
novelty_1,0.382508,0.0002
novelty_2,0.448534,0.0
novelty_m,0.436293,0.0


human ratings:
- almost always significant
- if not, barely off

ewm methods:
- most of the time, far from 0.05
- some occasional significance

minvec methods:
minvec:
- 4/8 sig
minvec_clust:
- 5/8 sig
minvec_min:
- 3/8 sig

## Comparing Fluency with Flexibility Results

In [155]:
def calculate_corrs_originality_flexibility(prompt_list, prompt):
    collapse_flexibility_df = flexibility_dict[prompt]
    collapse_fluency_df = fluency_dict[prompt]
    participant_avg_flexibility_df = get_flexibility_avg(collapse_flexibility_df)
    participant_avg_fluency_df = get_fluency_avg(collapse_fluency_df)
    combined_df = pd.merge(participant_avg_flexibility_df, participant_avg_fluency_df, how='inner', on = 'id')
    combined_df = combined_df.drop('id', axis=1)
    pval_df = calculate_pvalues(combined_df)
    corrs_df = combined_df.corr()
    corrs_df = pd.DataFrame(corrs_df.iloc[:4, 4:])
    pval_df = pd.DataFrame(pval_df.iloc[:4, 4:])
    pval_df.columns = ['fluency_pval']
    corr_pval_df = pd.concat([corrs_df, pval_df], axis=1)
    corr_pval_df = corr_pval_df[['fluency', 'fluency_pval']]
    corr_pval_df.index.rename('metrics', inplace=True)
    display(corr_pval_df)

In [156]:
print_per_prompt_results(fluency_dict, calculate_corrs_originality_flexibility)

<function calculate_corrs_originality_flexibility at 0x00000173B56CA700>
box


,fluency,fluency_pval
metrics,,
flex_1,0.443968,0.0
flex_2,0.443968,0.0
flex_m,0.443968,0.0
flex_method,0.420588,0.0


brick


,fluency,fluency_pval
metrics,,
flex_1,0.470881,0.0
flex_2,0.470881,0.0
flex_m,0.470881,0.0
flex_method,0.674312,0.0


chair


...

,fluency,fluency_pval
metrics,,
flex_1,0.429531,0.0
flex_2,0.429531,0.0
flex_m,0.429531,0.0
flex_method,0.614395,0.0


rope


,fluency,fluency_pval
metrics,,
flex_1,0.579038,0.0
flex_2,0.579038,0.0
flex_m,0.579038,0.0
flex_method,0.612169,0.0


shoe


,fluency,fluency_pval
metrics,,
flex_1,0.422699,0.0
flex_2,0.422699,0.0
flex_m,0.422699,0.0
flex_method,0.469671,0.0


always significant

## Comparing Novelty Method with Human Ratings

In [133]:
def calculate_corrs_novelty(prompt_list, prompt):
    collapse_flexibility_df = flexibility_dict[prompt]
    participant_avg_novelty_df = get_novelty_avg(collapse_novelty_df)
    participant_avg_novelty_df = participant_avg_novelty_df.drop('id', axis=1)
    pval_df = calculate_pvalues(participant_avg_novelty_df)
    corrs_df = participant_avg_novelty_df.corr()
    corrs_df = pd.DataFrame(corrs_df.iloc[:5, 5:])
    pval_df = pd.DataFrame(pval_df.iloc[:5, 5:])
    pval_df.columns = ['novelty_1_pval', "novelty_2_pval", 'novelty_m_pval']
    corr_pval_df = pd.concat([corrs_df, pval_df], axis=1)
    corr_pval_df = corr_pval_df[['novelty_1', "novelty_1_pval", 'novelty_2', 'novelty_2_pval', "novelty_m", 'novelty_m_pval']]
    corr_pval_df.index.rename('metrics', inplace=True)
    display(corr_pval_df)

In [134]:
print_per_prompt_results(originality_dict, calculate_corrs_novelty)

<function calculate_corrs_novelty at 0x00000173B5BFE5E0>
box


,novelty_1,novelty_1_pval,novelty_2,novelty_2_pval,novelty_m,novelty_m_pval
metrics,,,,,,
ewm,0.178069,0.095,0.303256,0.0039,0.266856,0.0115
ewm_clust,0.138428,0.1958,0.190230,0.0742,0.181521,0.0887
minvec,0.188921,0.0762,0.325012,0.0019,0.284985,0.0068
minvec_clust,0.239525,0.0238,0.265552,0.0119,0.277855,0.0084
minvec_min,-0.174450,0.102,-0.249300,0.0185,-0.234207,0.0272


brick


,novelty_1,novelty_1_pval,novelty_2,novelty_2_pval,novelty_m,novelty_m_pval
metrics,,,,,,
ewm,0.175395,0.1002,-0.098073,0.3605,0.039717,0.7117
ewm_clust,0.048370,0.6526,0.132887,0.2144,0.101862,0.3422
minvec,0.365414,0.0004,0.093329,0.3843,0.251937,0.0172
minvec_clust,0.269219,0.0107,0.393080,0.0001,0.369965,0.0004
minvec_min,-0.162385,0.1284,-0.249111,0.0186,-0.229982,0.0301


chair


...

,novelty_1,novelty_1_pval,novelty_2,novelty_2_pval,novelty_m,novelty_m_pval
metrics,,,,,,
ewm,0.590413,0.0,0.637952,0.0,0.641115,0.0
ewm_clust,0.491668,0.0,0.555130,0.0,0.545941,0.0
minvec,0.727145,0.0,0.757488,0.0,0.775352,0.0
minvec_clust,0.526805,0.0,0.581994,0.0,0.578492,0.0
minvec_min,-0.230517,0.0288,-0.139571,0.1895,-0.195039,0.0654


rope


,novelty_1,novelty_1_pval,novelty_2,novelty_2_pval,novelty_m,novelty_m_pval
metrics,,,,,,
ewm,0.233456,0.0268,0.292524,0.0051,0.285263,0.0064
ewm_clust,0.239983,0.0227,0.253600,0.0159,0.268707,0.0104
minvec,0.340263,0.001,0.310564,0.0029,0.355467,0.0006
minvec_clust,0.422946,0.0,0.388422,0.0002,0.443089,0.0
minvec_min,-0.366345,0.0004,-0.394580,0.0001,-0.414072,0.0


shoe


,novelty_1,novelty_1_pval,novelty_2,novelty_2_pval,novelty_m,novelty_m_pval
metrics,,,,,,
ewm,0.117001,0.2721,-0.038284,0.7202,0.041173,0.7
ewm_clust,0.121840,0.2526,0.112139,0.2927,0.122811,0.2488
minvec,0.180157,0.0893,-0.042893,0.6881,0.071841,0.501
minvec_clust,0.352379,0.0007,0.276066,0.0084,0.329811,0.0015
minvec_min,-0.214215,0.0426,-0.238353,0.0237,-0.237585,0.0241


ewm:
 - 4/8 sig novelty_m and novelty_1
 - 5/8 sig novelty_2
 
ewm_clust:
 - 4/8 sig novelty_m and novelty_2
 - 3/8 sig novelty_1

minvec:
- always significant for novelty_m
- 6/8 sig novelty_1 and novelty_2

minvec_clust:
- always significant for every rater

minvec_min:
- 5/8 sig novelty_m
- 4/8 sig novelty_1 and novelty_2

## Comparing Flexibility Method with Human Ratings

In [153]:
def calculate_corrs_flexibility(prompt_list, prompt):
    collapse_flexibility_df = flexibility_dict[prompt]
    participant_avg_flexibility_df = get_flexibility_avg(collapse_flexibility_df)
    participant_avg_flexibility_df = participant_avg_flexibility_df.drop('id', axis=1)
    pval_df = calculate_pvalues(participant_avg_flexibility_df)
    corrs_df = participant_avg_flexibility_df.corr()
    corrs_df = pd.DataFrame(corrs_df.iloc[:3, 3:])
    pval_df = pd.DataFrame(pval_df.iloc[:3, 3:])
    pval_df.columns = ['flex_method_pval']
    corr_pval_df = pd.concat([corrs_df, pval_df], axis=1)
    corr_pval_df.index.rename('metrics', inplace=True)
    display(corr_pval_df)

In [154]:
print_per_prompt_results(originality_dict, calculate_corrs_flexibility)

<function calculate_corrs_flexibility at 0x00000173B56CA4C0>
box


,flex_method,flex_method_pval
metrics,,
flex_1,0.397504,0.0001
flex_2,0.397504,0.0001
flex_m,0.397504,0.0001


brick


,flex_method,flex_method_pval
metrics,,
flex_1,0.277716,0.0084
flex_2,0.277716,0.0084
flex_m,0.277716,0.0084


chair


...

,flex_method,flex_method_pval
metrics,,
flex_1,0.281232,0.0073
flex_2,0.281232,0.0073
flex_m,0.281232,0.0073


rope


,flex_method,flex_method_pval
metrics,,
flex_1,0.336711,0.0012
flex_2,0.336711,0.0012
flex_m,0.336711,0.0012


shoe


,flex_method,flex_method_pval
metrics,,
flex_1,0.308043,0.0031
flex_2,0.308043,0.0031
flex_m,0.308043,0.0031


7/8 prompts sig